### Задание 1

* Импортируйте библиотеки pandas и numpy.
* Загрузите "Boston House Prices dataset" из встроенных наборов данных библиотеки sklearn..
* Разбейте эти датафреймы на тренировочные (X_train, y_train) и тестовые (X_test, y_test) с помощью функции train_test_split так, чтобы размер тестовой выборки
составлял 30% от всех данных, при этом аргумент random state должен быть равен 42.
* Создайте модель линейной регрессии под названием lr с помощью класса LinearRegression из модуля sklearn.linear_model.
* Обучите модель на тренировочных данных (используйте все признаки) и сделайте предсказание на тестовых

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import load_boston

In [3]:
boston = load_boston()

df = boston["data"]

In [4]:
X = pd.DataFrame(df, columns=boston["feature_names"])
y = pd.DataFrame(boston["target"], columns=["price"])

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

In [7]:
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

In [8]:
from sklearn.metrics import r2_score

r2_score(y_test, y_pred_lr)

0.7112260057484914

### Задание 2

* Создайте модель под названием model с помощью класса RandomForestRegressor из модуля
sklearn.ensemble.
* Сделайте агрумент n_estimators равным 1000, max_depth должен быть равен 12 и random_state сделайте равным 42.
* Обучите модель на тренировочных данных аналогично тому, как вы обучали модель LinearRegression, но при этом в метод fit вместо датафрейма y_train поставьте y_train.values[:, 0], чтобы получить из датафрейма одномерный массив Numpy,
так как для класса RandomForestRegressor в данном методе для аргумента y предпочтительно применение массивов вместо датафрейма.
* Сделайте предсказание на тестовых данных и посчитайте R2. Сравните с результатом из предыдущего задания.
* Напишите в комментариях к коду, какая модель в данном случае работает лучше

In [9]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(max_depth=12,
                               n_estimators=1000,
                               random_state=42)

In [10]:
model.fit(X_train, y_train.values[:, 0])

RandomForestRegressor(max_depth=12, n_estimators=1000, random_state=42)

In [11]:
y_pred_rfr = model.predict(X_test)

In [12]:
r2_score(y_test, y_pred_rfr)

0.87472606157312

Вывод: Модель, построенная на линейной регрессии, показала результат в 0,7112 по метрике $R^2$. В то время как модель RandomForestRegressor достигла результата в 0,8747. Это объясняется тем, что такая модель лучше работает со сложными нелинейными данными.

### *Задание 3

* Вызовите документацию для класса RandomForestRegressor,
* найдите информацию об атрибуте feature_importances_.

* С помощью этого атрибута найдите сумму всех показателей важности, 
* установите, какие два признака показывают наибольшую важность

In [13]:
print(model.__doc__, sep='\n')


    A random forest regressor.

    A random forest is a meta estimator that fits a number of classifying
    decision trees on various sub-samples of the dataset and uses averaging
    to improve the predictive accuracy and control over-fitting.
    The sub-sample size is controlled with the `max_samples` parameter if
    `bootstrap=True` (default), otherwise the whole dataset is used to build
    each tree.

    Read more in the :ref:`User Guide <forest>`.

    Parameters
    ----------
    n_estimators : int, default=100
        The number of trees in the forest.

        .. versionchanged:: 0.22
           The default value of ``n_estimators`` changed from 10 to 100
           in 0.22.

    criterion : {"squared_error", "absolute_error", "poisson"},             default="squared_error"
        The function to measure the quality of a split. Supported criteria
        are "squared_error" for the mean squared error, which is equal to
        variance reduction as feature selection cr

**feature_importances_** : ndarray of shape (n_features,)
        Значения импорта признаков на основе примесей.
        Чем выше, тем важнее признак.
        Важность признака вычисляется как (нормализованное)
        общее снижение критерия, вызванное этим признаком.  Это также
        известна как важность Джини.

        Предупреждение: импорт признаков на основе примесей может вводить в заблуждение для
        признаков с высокой кардинальностью (множество уникальных значений). См.
        :func:`sklearn.inspection.permutation_importance` в качестве альтернативы.

Посмотрим значения feature_importances_

In [14]:
model.feature_importances_

array([0.03167574, 0.00154252, 0.00713813, 0.00123624, 0.01426897,
       0.40268179, 0.01429864, 0.06397257, 0.00528122, 0.01152493,
       0.01808108, 0.01245085, 0.41584732])

In [15]:
# найдем сумму значений
sum_importances = model.feature_importances_.sum()
sum_importances

1.0

In [16]:
# выделим два наиболее важных признака
# отсортируем по убыванию пары (важность признака и название), и возьмем срез из 2 элементов
top2_features = sorted(zip(model.feature_importances_, X_train), reverse=True)[:2]
top2_features

[(0.4158473181914483, 'LSTAT'), (0.4026817857034993, 'RM')]

Вывод: Сумма показателей важности равна 1. Два наиболее важных показателя: 'LSTAT' - % более низкий статус населения (% бедноты???) = 0,4158, 'RM' - (кол-во комнат на 1 дом) = 0,4027
